# This is the first step in running corridor analysis for the Blueprint



Created by Amy Keister, last run by Amy Keister on 8/18/2023. It took 1 hour 32 minutes to run.

In [1]:
import os
import arcpy
import numpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb"

In [4]:
# define final outputs
#Out = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Resistance.tif"
Out= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Resistance"

In [5]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [6]:
# define additional inputs
subR= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintSubRgn.shp"
estuary= r"F:\GIS_DATA\WaterResources\NWI\Estuary.shp"
TNCLocalCon= r"F:\GIS_DATA\DecisionSupportTools\TNCTerrestrialResilient\Local_Connectedness\Local_connectedness_CONUSnew.tif"
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"
RoadX= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\WildlifeRoadCrossingsNCFL.gdb\NCandFLWildlifeRoadCrossingsNoAttributes"
# I had to do this buffer outside of the model, because it would not buffer all point inside the model
#RoadX210buff= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\NCandFLWildlifeRoadCrossings_Buffer210m.shp"
Road= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\WildlifeRoadCrossingsNCFL.gdb\RoadsAroundNCandFLWildlifeRoadCrossings"
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
WBD= r"F:\GIS_DATA\WaterResources\NHD\WBD_National_GDB\WBD_National_GDB.gdb\WBDHU2"

# Start Analysis

In [7]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb


# Define inland and nearshore area
Need to identify the area where the inland corridor analysis will occur. 

In [9]:
# make a copy of the subregion layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.CopyFeatures(subR, "subR", '', None, None, None)

In [10]:
# set code block for next step
codeblock = """
def Reclass(SubRgn):
    if SubRgn in ('Marine Gulf Stream', 'Marine Shelf and Extension'):
        return 'Marine'
    else:
        return 'Inland and near shore'
"""

In [11]:
# add and calculate a field that seperates marine from the inland and near shore
arcpy.management.CalculateField("subR", "SubRgn_III", "Reclass(!SubRgn!)", "PYTHON3", codeblock, "Text")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\subR'>

In [12]:
# dissolve into marine/inland and near shore marine
arcpy.management.Dissolve("subR", "subRD", "SubRgn_III", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\subRD'>

In [13]:
# pull out the inland and near shore 
arcpy.analysis.Select("subRD", "InlandNearShore", "SubRgn_III = 'Inland and near shore'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\InlandNearShore'>

# buffer inland and nearshore extent by 26.1 km
To try to allow corridors to connect to nearby areas outside the Southeast, we buffer it by 26.1 km 
This creates a problem in the coastal side, because we don't want to buffer into the marine environment. We take care of the problem by clipping back down to the WBD extent

In [14]:
# buffer inland and nearshore area by 26.1 km
arcpy.analysis.Buffer("InlandNearShore", "InlandNearShore_Buff", "26.1 Kilometers", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\InlandNearShore_Buff'>

In [15]:
# use HUC boundary to remove buffer area along the shore
# this still leaves extra buffer area in Mexico, but our data doesn't extend there, so it shouln't be a problem
arcpy.analysis.Clip("InlandNearShore_Buff", WBD, "InlandNearShore_Buff2", None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\InlandNearShore_Buff2'>

In [16]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandNearShore_Buff", snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.PolygonToRaster("InlandNearShore_Buff2", "OBJECTID", "InlandNearShoreBuffR", "CELL_CENTER", "NONE", SEraster)

# Use TNC local connectedness to fill in missing areas 
Our zonation results do not extend into the buffer area, so we are using TNC's local connectedness (from the resilence data) to fill in those areas. This data is from the original TNC download, so it does not go west of the Mississippi river.

In [17]:
# clip TNC local connectedness to buffered Inland raster extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask(TNCLocalCon, "InlandNearShoreBuffR"); out_raster.save("TNCLocalCon1")

In [18]:
# rescale the local connectedness so it will match the zonation, but I'm flipping it, making the low values high and the high values low
out_raster = arcpy.sa.RescaleByFunction("TNCLocalCon1", "LINEAR # # # # # #", 100, 1); out_raster.save("TNCLocalCon1Rescale")

In [19]:
# add 0.5 so that Int will round
out_raster = arcpy.sa.Plus("TNCLocalCon1Rescale", 0.5); out_raster.save("TNCLocalCon1Rescale1")

In [20]:
# make integer
out_raster = arcpy.sa.Int("TNCLocalCon1Rescale1"); out_raster.save("TNCLocalCon1RescaleInt1")

# Bring in Zonation results

In [21]:
# flip zonation results so that high is low and low is high to make resistance layer
out_raster = arcpy.sa.Minus(100, Zonation); out_raster.save("ZonationFlip")

In [22]:
# mosaic the zonation results with the TNC local connectedness, keeping zonation anywhere it exists
arcpy.management.MosaicToNewRaster(["TNCLocalCon1RescaleInt1", "ZonationFlip"], \
OutWorkspace, "Mosaic", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\Mosaic'>

# Clip back down to buffered inland area 
Need to remove the coastal areas

In [23]:
# use times to remove marine area from the mosaiced cost surface
out_raster = arcpy.sa.Times("Mosaic", "InlandNearShoreBuffR"); out_raster.save("MosaicInland")

# Identify estuaries to remove from inland analysis
In the past, we have seperated the terrestrial from the estuarine and marine for corridor analysis. 
We normally do not include deep water estuaries in the inland analysis
We normally include deep water estuaries in the marine corridor analysis

This year we are still doing that, but using a new esutary layer from NWI. 

In [28]:
# select out just the Estuarine and Marine Deepwater class from the NWI (not estuarine and Marine wetland)
arcpy.analysis.Select(estuary, "estuaryDeep", "WETLAND_TY = 'Estuarine and Marine Deepwater'")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\estuaryDeep'>

In [29]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.PolygonToRaster("estuaryDeep", "OBJECTID", "EstuaryDeepR", "CELL_CENTER", "NONE", SEraster)

In [30]:
# reclassify so I can use a times to remove deep estuaries
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify("EstuaryDeepR", "Value", "1 10000000 NODATA;NODATA 1", "DATA"); out_raster.save("EstuaryDeepR1")

In [31]:
# use times to remove estuarine area from the mosaiced cost surface
out_raster = arcpy.sa.Times("MosaicInland", "EstuaryDeepR1"); out_raster.save("MosaicInlandNoEst")

In [32]:
# use times to remove estuarine area from the inland mask (will be used to help build inland hubs)
out_raster = arcpy.sa.Times("InlandNearShoreBuffR", "EstuaryDeepR1"); out_raster.save("InlandNearShoreBuffRNoEst")

# Burn 90 meter NLCD urban in to try to influence corridors to go around


In [33]:
# Make a raster where developed, high or medium intensity is 150 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="MosaicInlandNoEst", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 150, '', "Value IN (23, 24)"); out_raster.save("DevHM")

In [34]:
# resample
arcpy.management.Resample("DevHM", "DevHM_90m", "90 90", "MAJORITY")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\DevHM_90m'>

In [35]:
# Combine urban with the mosaiced zonation results, using a maximum value so that these roads are very hard to cross
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="MosaicInlandNoEst", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.CellStatistics(["MosaicInlandNoEst", "DevHM_90m"], "MAXIMUM", 'DATA'); out_raster.save("MosaicInlandNoEstUrb")

# make known wildlife road crossings easy to move through
Right now I only have a few known road crossings in eastern North Carolina, but I'm looking for more

In [36]:
# make a copy of the road crossing data because it is being stupid
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.CopyFeatures(RoadX, "RoadX", '', None, None, None)

In [37]:
# buffer road crossings by 210 meters
arcpy.analysis.Buffer("RoadX", "RoadX210buff", "210 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\RoadX210buff'>

In [39]:
# add and calculate field to use to raster conversion
arcpy.management.CalculateField("RoadX210buff", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\RoadX210buff'>

In [42]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster("RoadX210buff", "raster", "RoadXR", "CELL_CENTER", "NONE", SEraster)

In [43]:
# now bring in roads around wildlife road crossings, we want to make these extra hard to move through in the 
# resistance raster
# buffer roads around crossings by 180 meters
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer(Road, "Road", "180 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [44]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster("Road", "BUFF_DIST", "RoadR", "CELL_CENTER", "NONE", SEraster)

In [45]:
# reclassify roads near wildlife road crossings and give a value of 150
out_raster = arcpy.sa.Reclassify("RoadR", "Value", "180 150", "DATA"); out_raster.save("RoadR2")

In [46]:
# maybe also do a larger buffer and make it low but not zero resistance
# buffer road crossings by 450 meters
arcpy.analysis.Buffer(RoadX, "RoadX2", "900 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\RoadX2'>

In [47]:
# erase road area from the larger buffered road crossing
arcpy.analysis.Erase("RoadX2", "Road", "RoadX2_Erase", None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\RoadX2_Erase'>

In [48]:
# add field for converting to raster
arcpy.management.CalculateField(
    in_table="RoadX2_Erase",
    field="raster",
    expression="10",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\A_ConInlandResistance.gdb\\RoadX2_Erase'>

In [49]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster("RoadX2_Erase", "raster", "RoadX2R", "CELL_CENTER", "NONE", SEraster)

In [50]:
# reclassify larger buffer and give a value of 10
#out_raster = arcpy.sa.Reclassify("RoadX2R", "Value", "0 10", "DATA"); out_raster.save("RoadX3R")

In [51]:
# Combine roads with the mosaiced zonation results, using a maximum value so that these roads are very hard to cross
out_raster = arcpy.sa.CellStatistics(["MosaicInlandNoEstUrb", "RoadR2"], "MAXIMUM", 'DATA'); out_raster.save("MosaicInRoad")

In [52]:
# Combine above raster with the road crossing data, using a minimum value so that the areas with road crossings are
# easier to cross
out_raster = arcpy.sa.CellStatistics(["MosaicInRoad","RoadX2R", "RoadXR"], "MINIMUM", 'DATA'); out_raster.save("MosaicInRoadX")

In [53]:
# use times to remove the urban and road areas that fall outside extent
out_raster = arcpy.sa.Times("MosaicInRoadX", "InlandNearShoreBuffRNoEst"); out_raster.save("MosaicInRoadXa")

# resample because 30 meter is too large to use with Linkage Mapper over this size area

In [54]:
# might need to resample to 60 or 90 meter pixels like I had to do last year
# I tried all the options (Nearest,Majority, Bilinear, Cubic) it is a tough call
# the bilinear did a little better job retaining linear features
arcpy.management.Resample("MosaicInRoadXa", Out, "90 90", "BILINEAR")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ba_Con_LM_Input.gdb\\Resistance'>

In [55]:
end = time.time()
print(end - start)

5552.595414876938


# bring in potential corridors
In the 2021 South Atlantic Blueprint, we used a new method where we use corridors from other projects to try to help keep corridors more stable. The idea is that we use our old corridors and corridors from other projects and use them to limit the resistance raster to try to get corridors to move through them.

Since we didn't have previous corridors for the whole Base Blueprint area, we didn't do that this year. We expect to pick this back up in 2023 base blueprint.

In [ ]:
# Make eastern wildway core a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(EWCore, "Id", "EWCore", 30)

In [ ]:
# Make eastern wildway corridor a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(EWCorr, "Id", "EWCorr", 30)

In [ ]:
# Make SA 2020 hub a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(SA20Hub, "GRIDCODE", "SA20Hub", 30)

In [ ]:
# Select out top portion of SA 2020 linkagemapper corridor output
# I'm picking 500000 as a cutoff, It may be helpful to move that around in testing
# could try 1000000 or 1200000
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    out_raster = arcpy.sa.Con(SA20Cor, 1, None, "Value < 500000"); out_raster.save("SA20Cor")

In [ ]:
# combine potential cores/corridors
#out_raster = arcpy.sa.CellStatistics(["EWCore","EWCorr","SA20Hub", "SA20Cor"], "MAXIMUM", 'DATA'); out_raster.save("Prev")

In [ ]:
#reclassify 
#out_raster = arcpy.sa.Reclassify("Prev", "Value", "0 500 1;NODATA 0", "DATA"); out_raster.save("Prev1")

In [ ]:
# change NODATA to zero so expand will work
#out_raster = arcpy.sa.Reclassify("Prev", "Value", "1 1; 0 0; NODATA 0", "DATA"); out_raster.save("Top10zROW2")

In [ ]:
# expand to get rid of small gaps
#out_raster = arcpy.sa.Expand("Prev1", 2, 1, ''); out_raster.save("Prev2")

In [ ]:
# shrink to bring it back to size
#out_raster = arcpy.sa.Shrink("Prev2", 2, 1, ''); out_raster.save("Prev3")

In [ ]:
# change 0 to NoData
#out_raster = arcpy.sa.Reclassify("Prev3", "Value", "1 1;0 NODATA", "DATA"); out_raster.save("Prev4")

# limit resistance raster to areas around previous corridors

In [ ]:
# limit resistance raster to areas around previous corridors, this is the most dramatic option and does not allow
# linkagemapper to even consider areas that were not around previous corridors
#with arcpy.EnvManager(outputCoordinateSystem=sr, mask="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    out_raster = arcpy.sa.ExtractByMask("MosaicInRoadX", "Prev4"); out_raster.save("ResistanceTest7")

# clip to test area

In [ ]:
# clip to a test area so I can see how it is working
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Pextent):
#    out_raster = arcpy.sa.Times("ResistanceTest7_90mBilinear", 1); out_raster.save("ResistanceTest7_90mBilinearTest")

# Explore smoothing out the estuaries layer
The estuaries layer is very detailed and has lots of islands created by narrow estuaries and canals. 

Do we want inland corridors to be able to go to those islands? If so, we need to connect them to the mainland by smoothing out and removing the narrow "estuaries"

I didn't use this in the test, but I'm keeping it in case we need it after testing

In [ ]:
# expand by 1 cells to try to close off rivers but leave estuaries
#out_raster = arcpy.sa.Expand("InlandR", 1, [1], "MORPHOLOGICAL"); out_raster.save("InlandREx1")

In [ ]:
# shrink above raster to return to something close to previous shoreline, but it will be smoothed some
#out_raster = arcpy.sa.Shrink("InlandREx1", 1, [1]); out_raster.save("InlandRExSh1")

In [ ]:
# expand by 2 cells to try to close off rivers but leave estuaries
#out_raster = arcpy.sa.Expand("InlandR", 2, [1], "MORPHOLOGICAL"); out_raster.save("InlandREx2")

In [ ]:
# shrink above raster to return to something close to previous shoreline, but it will be smoothed some
#out_raster = arcpy.sa.Shrink("InlandREx2", 2, [1]); out_raster.save("InlandRExSh2")

In [ ]:
# expand by 3 cells to try to close off rivers but leave estuaries
#out_raster = arcpy.sa.Expand("InlandR", 3, [1], "MORPHOLOGICAL"); out_raster.save("InlandREx3")

In [ ]:
# shrink above raster to return to something close to previous shoreline, but it will be smoothed some
# this cuts off a lot of estuaries at the neck and makes isolated areas
#out_raster = arcpy.sa.Shrink("InlandREx3", 3, [1]); out_raster.save("InlandRExSh3")

In [ ]:
# expand by 4 cells to try to close off rivers but leave estuaries
# this cuts off a lot of estuaries at the neck and makes isolated areas
#out_raster = arcpy.sa.Expand("InlandR", 4, [1], "MORPHOLOGICAL"); out_raster.save("InlandREx4")

In [ ]:
# shrink above raster to return to something close to previous shoreline, but it will be smoothed some
#out_raster = arcpy.sa.Shrink("InlandREx4", 4, [1]); out_raster.save("InlandRExSh4")